<table>
 <tr align=left><td><img align=left src="./images/CC-BY.png">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license. (c) Kyle T. Mandli</td>
</table>

In [ ]:
from __future__ import print_function

%matplotlib inline
import numpy
import matplotlib.pyplot as plt
import matplotlib.animation
from IPython.display import HTML

import os
import pyclaw
path = os.path.join(os.environ.get("CLAW", os.getcwd()), "pyclaw", "fvmbook", "chap6")
os.chdir(path)
import advection

# High-Resolution Methods

The general approach we will take to augmenting our wave propagation methodology is to add correction terms in the following way:
$$
    Q^{n+1}_i = Q^n_i - \frac{\Delta t}{\Delta x} \left (\mathcal{A}^+_{i-1/2} + \mathcal{A}^-_{i+1/2}  \right) - \frac{\Delta t}{\Delta x} \left (\widetilde{F}_{i+1/2} - \widetilde{F}_{i-1/2} \right)
$$
where here we have written these corrections as fluxes.

## Lax-Wendroff Method

The first approach we will investigate is the Lax-Wendroff method derived from a Taylor series approach.  Recall that the taylor series of our solution has the form
$$
    q(x, t_{n+1}) = q(x, t_n) + \Delta t q_t(x, t_n) + \frac{1}{2} \Delta t^2 q_{tt}(x, t_n) + \ldots
$$
If we consider solving the linear advection problem $q_t + A q_x = 0$ and assume *smooth solutions* then we can replace time derivatives with spatial derivaties in our taylor series above where $q_t = -A q_x$ to find
$$
    q(x, t_{n+1}) = q(x, t_n) - \Delta t A q_x(x, t_n) + \frac{1}{2} \Delta t^2 q_{tt}(x, t_n) + \ldots
$$

Taking the second time derivative of $q_t = -A q_x$ leads to
$$\begin{aligned}
    \frac{\partial}{\partial t} (q_t) =& \frac{\partial}{\partial t}(-A q_x) \\
    q_{tt} =& -A (q_x)_t \\
    q_{tt} =&  A^2 q_{xx}.
\end{aligned}$$
Plugging this in again into our Taylor series we now have
$$
    q(x, t_{n+1}) = q(x, t_n) - \Delta t A q_x(x, t_n) + \frac{1}{2} \Delta t^2 A^2 q_{xx}(x, t_n) + \ldots
$$

Replacing the spatial derivatives with second order finite differences then leads us to the Lax-Wendroff method
$$
    Q^{n+1}_i = Q^n_i - \frac{\Delta t}{2 \Delta x} (Q^n_{i+1} - Q^n_{i-1}) + \frac{1}{2} \left(\frac{\Delta t}{\Delta x}\right)^2 A^2 (Q^n_{i+1} - Q^n_i + Q^n_{i-1}).
$$

This can again be interpreted as a finite volume method with the flux function defined as
$$
    F^n_{i-1/2} =\frac{1}{2} A (Q^n_{i-1} + Q^n_i) - \frac{1}{2} \frac{\Delta t}{\Delta x} A^2 (Q^n_i - Q^n_{i-1})
$$
As mentioned before this looks like the unstable flux with a diffusive correction term.  From a method of lines and absolute stablity point-of-view this is exactly what pushed the eigenvalues into the absolute stability region of forward Euler.

In [ ]:
def q_true(t, IC='gauss_square'):
    # True Solution
    x_true = numpy.linspace(0.0, 1.0, 1000)
    x_shift = x_true - t
    x_shift = numpy.mod(x_shift, 1.0)
    if IC == 'gauss_square':
        x1 = 0.6; x2 = 0.8
        beta=200.; x0=0.3; mx=100
        q_true = numpy.exp(-beta * (x_shift-x0)**2) + (x_shift>0.6)*(x_shift<0.8)
    elif IC == 'wavepacket':
        beta=100.; x0=0.5; mx=100
        q_true = numpy.exp(-beta * (x_shift-x0)**2) * sin(80.*x_shift)
    return q_true

def advection_animation(solver_order=2, limiters=0, IC='gauss_square'):
    """
    Solve the pendulum problem with the method defined above using nsteps time steps.
    Create an animation showing the swinging pendulum, using the solution every
    plot_interval time steps.
    """
   
    # compute the solution with the method define above:
    claw = advection.fig_61_62_63(solver_order=solver_order, limiters=limiters)
    claw.keep_copy = True
    claw.run()
    x = claw.frames[0].grid.dimensions[0].centers
    x_true = numpy.linspace(0.0, 1.0, 1000)
    
    fig = plt.figure()
    axes = plt.subplot(1, 1, 1)
    axes.set_xlim((x[0], x[-1]))
    axes.set_ylim((-0.1, 1.1))
    axes.set_title("Lax-Wendroff")
    
    def init():
        axes.set_xlim((x[0], x[-1]))
        axes.set_ylim((-0.1, 1.1))
        computed_line, = axes.plot(x[0], claw.frames[0].q[0, :][0], 'ro')
        exact_line, = axes.plot(x_true[0], q_true(0.0)[0], 'k')
        return (computed_line, exact_line)
    
    computed_line, exact_line = init()
    
    def fplot(n):
        computed_line.set_data([x,], [claw.frames[n].q[0, :]])
        exact_line.set_data([x_true], [q_true(claw.frames[n].t)])
        return (computed_line, exact_line)

    frames_to_plot = range(0, len(claw.frames))
    plt.close(fig)
    return matplotlib.animation.FuncAnimation(fig, fplot, frames=frames_to_plot, interval=100,
                                   blit=True, init_func=init, repeat=False)

In [ ]:
HTML(advection_animation(solver_order=2, limiters=0, IC='gauss_square').to_jshtml())

## Beam-Warming Method
Another approach to second order accuracy is to instead use one-sided or upwinded derivative approximations.  With the same Taylor series approach if we instead replace the spatial derivatives with upwinded differences we then have
$$\begin{aligned}
    q_x(x_i,t_n) &= \frac{1}{2\Delta x} [3 q(x_i, t_n) - 4q(x_{i-1}, t_n) + q(x_{i-2}, t_n)] + \mathcal{O}(\Delta x^2) \\
    q_{xx}(x_i,t_n) &= \frac{1}{\Delta x^2} [q(x_i, t_n) - 2q(x_{i-1}, t_n) + q(x_{i-2}, t_n)] + \mathcal{O}(\Delta x^2) \\
\end{aligned}$$
leading to
$$
    Q^{n+1}_i = Q^n_i - \frac{\Delta t}{2 \Delta x} A (3 Q^n_i - 4 Q^n_{i-1} + Q^n_{i-2} ) + \frac{1}{2} \left(\frac{\Delta t}{\Delta z} \right)^2 A^2 (Q^n - 2 Q^n_{i-1} + Q^n_{i-2})
$$
or in flux form
$$
    F^n_{i-1/2} = A Q^n_{i-1} + \frac{1}{2} A \left ( 1 - \frac{\Delta t}{\Delta x} A \right) (Q^n_{i-1} - Q^n_{i-2}).
$$

## Preview of Limiters

Given that higher order methods work well in smooth regions while lower order methods work better in regions where high gradients or discontinuities exist we will pursue a way to combine the advantages of both and decide dynamically what type of method to use.  This is the primary role of a limiter.

As a first examination of this idea consider the numerical flux for the Lax-Wendroff method rewritten in the form
$$
    F^n_{i-1/2} = (A^- Q^n_i + A^+Q^n_{i-1}) + \frac{1}{2} |A| \left ( I - \frac{\Delta t}{\Delta x} |A| \right) (Q^n_i - Q^n_{i-1})
$$
This is therefore the upwind flux with a correction term added similar to how we suggested in the beginning of this lecture.

We can also rewrite the previous flux then as
$$\begin{aligned}
    F^n_{i-1/2} &= (A^- Q^n_i + A^+Q^n_{i-1}) + \frac{1}{2} |A| \left ( I - \frac{\Delta t}{\Delta x} |A| \right) (Q^n_i - Q^n_{i-1}) \\
    &=\mathcal{F}_L(Q_{i-1}, Q_i) + \Phi^n_{i-1/2} [\mathcal{F}_H(Q_{i-1}, Q_i) - \mathcal{F}_L(Q_{i-1}, Q_i)]
\end{aligned}$$

In this form if $\Phi^n_{i-1/2} = 0$ then this reduces to the low-order method and if $\Phi^n_{i-1/2} = 1$ then we get the high-order method.

In [ ]:
### Compare several methods for advecting a Gaussian and square wave.
### 
### The settings coded here are for Figure 6.1(a).
### For Figure 6.1(b), set solver.order=2.
### For Figure 6.2(a), set solver.order=2 and solver.limiters = pyclaw.limiters.tvd.minmod (1)
### For Figure 6.2(b), set solver.order=2 and solver.limiters = pyclaw.limiters.tvd.superbee (2)
### For Figure 6.2(c), set solver.order=2 and solver.limiters = pyclaw.limiters.tvd.MC (4)

# HTML(advection_animation(solver_order=2, limiters=0, IC='gauss_square').to_jshtml())
HTML(advection_animation(solver_order=2, limiters=1, IC='gauss_square').to_jshtml())
# HTML(advection_animation(solver_order=2, limiters=2, IC='gauss_square').to_jshtml())
# HTML(advection_animation(solver_order=2, limiters=4, IC='gauss_square').to_jshtml())

## The REA Algorithm with Piecewise Linear Reconstruction

In the REA algorithm we have the following steps:

1. Reconstruct
1. Evolve
1. Average

One of the ways to understand higher-oder methods is to modify the *reconstruct* step to higher-order polynomials.

From the cell averages $Q^n_i$ we need to construct piecewise linear functions such that
$$
    \widetilde{q}^n(x, t_n) = Q^n_i + \sigma^n_i (x - x_i) \quad \text{for } x_{i-1/2} \leq x \leq x_{i+1/2}
$$
assuming that x_i is the cell center.  We now need to figure out how to determine the slope $\sigma^n_i$.

If we assume $u > 0$ we can compute for linear advection $q_t + u q_x = 0$ that the update formula that correct solution to this problem should be
$$\begin{aligned}
    Q^{n+1}_i &= \frac{u \Delta t}{\Delta x} \left(Q^n_{i-1} + \frac{1}{2} (\Delta x - u \Delta t) \sigma^n_{i-1} \right) + \left(1 - \frac{u \Delta t}{\Delta x} \right) \left ( Q^n_i - \frac{1}{2} u \Delta t \sigma^n_i \right) \\
    &= Q^n_i - \frac{u \Delta t}{\Delta x}(Q^n_i - Q^n_{i-1} ) - \frac{1}{2} \frac{u \Delta t}{\Delta x} (\Delta x - u \Delta t) (\sigma^n_i - \sigma^n_{i-1})
\end{aligned}$$

## Choice of Slopes

We still have yet to specify which slope we would like to use for $\sigma^n_i$.  For a linear reconstruction we have three options:
$$\begin{aligned}
    &\text{Centered} & & \sigma^n_i = \frac{Q^n_{i+1} - Q^n_{i-1}}{2 \Delta x} && \text{(Fromm)} \\
    &\text{Upwind Slope} & & \sigma^n_i = \frac{Q^n_{i} - Q^n_{i-1}}{\Delta x} && \text{(Beam-Warming)} \\
    &\text{Downwind Slope} & & \sigma^n_i = \frac{Q^n_{i+1} - Q^n_{i}}{\Delta x} && \text{(Lax-Wendroff)}
\end{aligned}$$
At this point it is not clear exactly which slope might be the best.

## Oscillations

One of the key observations from the Lax-Wendroff or Beam-Warming methods is that spurious osciallations end up being created.  From modified equation analysis this is not surprising given that the leading error terms are dispersive in nature.  In the context of hyperbolic PDEs we would like to avoid these oscillations as they are generally not indicative of true solutions.  In fact, at a discontinuity we know that in many instances that the solution should dissipating energy or smoothing the solution physically.  Therefore we will seek to remove these oscillations from our solutions.

## Total Variation

To provide a concrete way to *limit* the slope we will introduce a mathematical way to measure oscillations.

The **total variation** for a grid function $Q$ is defined as
$$
    \text{TV}(Q) = \sum^\infty_{i=-\infty} |Q_i - Q_{i-1} |
$$

We can also define for a function this metric as 
$$
    \text{TV}(q(x)) = \sup \sum^N_{j=1} |q(\xi_j) - q(\xi_{j-1}) |
$$
Another definition is
$$
    \text{TV}(q) = \limsup_{\epsilon \rightarrow 0} \frac{1}{\epsilon} \int^\infty_{-\infty} |q(x) - q(x - \epsilon)|dx
$$
which reduces to
$$
    \text{TV}(q) = \int^\infty_{-\infty} |q'(x)|dx
$$
if $q$ is differentiable.

A two-level method is called **total variation diminitioning** (TVD) if, for any set of data $Q^n$, the values $Q^{n+1}$ computed by the method satisfies
$$
    \text{TV}(Q^{n+1}) \leq \text{TV}(Q^n).
$$

One important consequence of a TVD method is that if data is intially monotone in the sense that
$$
    Q^n_i \leq Q^n_{i+1} \text{    or    } Q^n_i \geq Q^n_{i+1} \quad \forall i
$$
then this will hold for all future time.  This leads us to the next definition.

A method is called **monotonicity-preserving** if 
$$
    Q^n_i \geq Q^n_{i+1} \forall i \Rightarrow Q^{n+1}_i \geq Q^{n+1}_{i+1} \forall i
$$
Any TVD method is monotonicity preserving.

How then can we derive a TVD method?  If we follow the REA algorithm then in the reconstruction phase we can require
$$
    \text{TV}(\widetilde{q}^n(x, t_n)) \leq \text{TV}(Q^n)
$$
so that our resulting evolved function should reamin TVD.  This can be clearly seen as the evolve step will also maintain this relationship as will the averaging.

## Slope-Limiter Methods

Now that we have established some basis as to how we may want to reconstruct functinos let us return to the question of reconstructing the slopes $\sigma$.

One choice of slope that maintains second-order accuracy and the TVD property is the **minmod slope** defined as 
$$
    \sigma^n_i = \text{minmod}\left( \frac{Q^n_i - Q^n_{i-1}}{\Delta x}, \frac{Q^n_{i+1} - Q^n_{i}}{\Delta x} \right)
$$
where
$$
    \text{minmod}(a, b) = \left \{ \begin{aligned}
        a & & \text{if } |a| < |b| \text{ and } ab > 0 \\
        b & & \text{if } |b| < |a| \text{ and } ab > 0 \\
        0 & & \text{if } ab \leq 0 \\
    \end{aligned} \right .
$$

Compositions of the minmod function also form a number of slope choices.  One in particular is the **monotonized central-difference limite** (MC)
$$
    \sigma^n_i = \text{minmod}\left( \left (\frac{Q^n_{i+1} - Q^n_{i-1}}{2 \Delta x} \right ), 2\left(\frac{Q^n_i - Q^n_{i-1}}{\Delta x} \right), 2 \left (\frac{Q^n_{i+1} - Q^n_{i}}{\Delta x} \right ) \right )
$$
Note that these still represent finite difference approximations.

## Flux Formulation with Piecewise Linear Reconstruction

Using these reconstructed slopes how does this now impact our flux formulations?  Let us turn to the basic definition of a flux function then and see how it is impacted by our reconstruction.

$$\begin{aligned}
    F^n_{i-1/2} &= \frac{1}{\Delta t} \int^{t_{n+1}}_{t_n} u \widetilde{q}^n(x_{i-1/2}, t) dt \\
    &= \frac{1}{\Delta t} \int^{t_{n+1}}_{t_n} u \widetilde{q}^n(x_{i-1/2} - u(t - t_n, t_n) dt \\
    &= \frac{1}{\Delta t} \int^{t_{n+1}}_{t_n} u \left[ Q^n_{i-1} + (x_{i-1/2} - u (t - t_n) - x_{i-1} ) \sigma^n_{i-1} \right] dt \\
    &= u Q^n_{i-1} + \frac{1}{2} u (\Delta x - u \Delta t) \sigma^n_{i-1} \\
\end{aligned}$$
Plugging this into our original updating formula leads to
$$
    Q^{n+1}_i = Q^n_i - \frac{u \Delta t}{\Delta x} (Q^n_i - Q^n_{i-1} ) - \frac{1}{2} \frac{u \Delta t}{\Delta x} (\Delta x  - u \Delta t) (\sigma^n_i - \sigma^n_{i-1})
$$

We can also think instead of limiting the fluxes between cells to update our first-order methods to higher-order.  If we define
$$
    \Delta Q^n_{i-1/2} \equiv Q^n_i - Q^n_{i-1}
$$
and then define a flux such that
$$
    F^n_{i-1/2} = u^- Q^n_i + u^+ Q^n_{i-1} + \frac{1}{2} |u| \left ( 1- \left |\frac{u \Delta t}{\Delta x} \right |\right) \delta^n_{i-1/2}
$$
where
$$
    \delta^n_{i-1/2} \equiv \text{limited} (\Delta Q^n_{i-1/2})
$$

## Flux Limiters

We can now start to make connections between our desire for high-order methods, limiting, and TVD.  Lax-Wendroff for instance is simply interpretable as the piecewise linear reconstruction in the most straight-forward way as as central difference.  Let us meld the rest of the methods and others by generalizing this idea a bit.

Let
$$
    \delta^n_{i-1/2} = \phi(\theta^n_{i-1/2}) \Delta Q^n_{i-1/2}
$$
where
$$
    \theta^n_{i-1/2} = \frac{\Delta Q^n_{I-1/2}}{\Delta Q^n_{i-1/2}}
$$
and
$$
    I = \left \{ \begin{aligned}
        i - 1 & & \text{if } u > 0 \\
        i + 1 & & \text{if } u < 0
    \end{aligned} \right .
$$
Basically we are "measuring" the smoothness of the function $Q$ using $\theta$.  We then want the function $\phi$ to "limit" the slope based on the smoothness of $Q$.

One interesting case is to choose
$$
    \phi(\theta) = \theta
$$
where
$$
    \delta^n_{i-1/2} = \left( \frac{\Delta Q^n_{I-1/2}}{\Delta Q^n_{i-1/2}} \right) \Delta Q^n_{i-1/2} = \Delta Q^n_{I-1/2}
$$
which reduces to the upwind scheme and therefore the Beam-Warming method.

Given this framework we can then define the following set of methods:

Linear Methods:
$$\begin{aligned}
    \text{upwind:} & & \phi(\theta) &= 0 \\
    \text{Lax-Wendroff:} & & \phi(\theta) &= 1 \\
    \text{Beam-Warming:} & & \phi(\theta) &= \theta \\
    \text{Fromm:} & & \phi(\theta) &= \frac{1}{2} (1 + \theta) \\
\end{aligned}$$

High-Resolution (non-linear) Methods:
$$\begin{aligned}
    \text{minmod:} & & \phi(\theta) &= \text{minmod}(1, \theta) \\
    \text{superbee:} & & \phi(\theta) &= \max(0, \min(1, 2\theta), \min(2, \theta) \\
    \text{MC:} & & \phi(\theta) &= \max(0, \min((1+\theta)/2, 2, 2 \theta)\\
    \text{van Leer:} & & \phi(\theta) &= \frac{\theta + |\theta|}{1 + |\theta|} \\
\end{aligned}$$

## TVD Limiters

Returning back to our desire for having a TVD scheme we need to ask the question, which of our limiters will make our scheme TVD?  This is at times for linear limiters easy to check but we need something a bit more powerful for the non-linear limiters we have introduced.

**Harten**: Consider a general method of the form
$$
    Q^{n+1}_i = Q^n_i - C^n_i (Q^n_i - Q^n_{i-1}) + D^n_i (Q^n_{i+1} - Q^n_i)
$$
over one time step, where the coefficients $C^n_{i-1}$ and $D^n_i$ are arbitrary (which can depend on values of Q^n in particular).  Then
$$
    \text{TV}(Q^{n+1}) \leq \text{TV}(Q^n)
$$
provided the following conditions are satisfied
$$\begin{aligned}
    C^n_{i-1} \geq 0 & & \forall i \\
    D^n_{i} \geq 0 & & \forall i \\
    C^n_{i} + D^n_i \leq 1 & & \forall i \\
\end{aligned}$$

## Methods for Systems

## Wave-Propagation Implementation

## Non-Linear Systems